## 1. Installation
Ensure you have Python installed. You can install Keras with the following command:
```bash
pip install keras tensorflow
```

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import os
from PIL import Image

## 2. Basics of Tensors

In [ ]:
# Creating Tensors with TensorFlow
x = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
y = tf.random.uniform((2, 2))
z = tf.zeros((2, 2))
w = tf.ones((2, 2))

print(x)
print(y)
print(z)
print(w)

# Tensor Operations
add_result = x + y
mul_result = x * y
dot_result = tf.linalg.matmul(x, y)

print(add_result)
print(mul_result)
print(dot_result)

## 3. Data Loading and Preprocessing

In [ ]:
# Using ImageDataGenerator for data loading and augmentation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation')

## 4. Building Neural Networks

In [ ]:
# Define a simple neural network using Sequential API
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.summary()

## 5. Loss Function and Optimizer

In [ ]:
# Compile the model
model.compile(optimizer=SGD(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 6. Training the Network

In [ ]:
# Train the model
history = model.fit(train_generator, epochs=5, validation_data=validation_generator)

## 7. Evaluation

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Accuracy: {accuracy * 100}%')

## 8. Saving and Loading Models

In [ ]:
# Save the model
model.save('simple_model.h5')

# Load the model
new_model = tf.keras.models.load_model('simple_model.h5')
new_model.summary()

## 9. Moving to GPU

In [ ]:
# Ensure the operations are running on the GPU
with tf.device('/GPU:0'):
    x = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
    y = tf.random.uniform((2, 2))
    z = tf.zeros((2, 2))
    w = tf.ones((2, 2))

    print(x)
    print(y)
    print(z)
    print(w)

## 10. Advanced Topics

### Custom Data Reading

In [ ]:
# Custom data reading
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, csv_file, root_dir, batch_size=32, dim=(150, 150), n_channels=3, n_classes=10, shuffle=True):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.annotations) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_annotations = [self.annotations.iloc[k] for k in indexes]
        X, y = self.__data_generation(batch_annotations)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.annotations))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_annotations):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        for i, annotation in enumerate(batch_annotations):
            img_name = os.path.join(self.root_dir, annotation[0])
            image = Image.open(img_name)
            image = image.resize(self.dim)
            X[i,] = np.array(image) / 255.0
            y[i] = annotation[1]
        return X, tf.keras.utils.to_categorical(y, num_classes=self.n_classes)

# Usage example
custom_data_gen = CustomDataGenerator(csv_file='data/labels.csv', root_dir='data/images', batch_size=32)

### Scheduler

In [ ]:
# Using a learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

model.compile(optimizer=SGD(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training with the scheduler
history = model.fit(train_generator, epochs=5, validation_data=validation_generator)

### Data Augmentation

In [ ]:
# Data augmentation
data_augmentation = Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

# Apply data augmentation in the dataset
train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=data_augmentation)
train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training')

### Custom Data Augmentation

In [ ]:
# Custom data augmentation
def custom_augmentation(image):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.adjust_contrast(image, 2)
    return image

train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=custom_augmentation)
train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training')

### Custom Training Phase

In [ ]:
# Custom training loop
def custom_train_model(model, train_generator, val_generator, epochs):
    for epoch in range(epochs):
        print(f'Starting epoch {epoch+1}/{epochs}')
        for step, (x_batch_train, y_batch_train) in enumerate(train_generator):
            with tf.GradientTape() as tape:
                logits = model(x_batch_train, training=True)
                loss_value = tf.keras.losses.sparse_categorical_crossentropy(y_batch_train, logits)
            grads = tape.gradient(loss_value, model.trainable_weights)
            model.optimizer.apply_gradients(zip(grads, model.trainable_weights))
            if step % 100 == 0:
                print(f'Epoch {epoch+1} Step {step}: Loss = {loss_value.numpy().mean()}')

        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        for x_batch_val, y_batch_val in val_generator:
            val_logits = model(x_batch_val, training=False)
            val_accuracy.update_state(y_batch_val, val_logits)
        print(f'Validation accuracy: {val_accuracy.result().numpy()}')

# Usage example
model.compile(optimizer=SGD(learning_rate=0.01), loss='sparse_categorical_crossentropy')
custom_train_model(model, train_generator, validation_generator, epochs=5)

### Custom Testing Phase

In [ ]:
# Custom testing loop
def custom_test_model(model, test_generator):
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    for x_batch_test, y_batch_test in test_generator:
        test_logits = model(x_batch_test, training=False)
        test_accuracy.update_state(y_batch_test, test_logits)
    print(f'Test accuracy: {test_accuracy.result().numpy()}')

# Usage example
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'data/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

custom_test_model(model, test_generator)